In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import utils
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


rawname_ = 'S01_off_hold'
rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'
#rawname_ = 'S02_on_hold' 
#rawname_ = 'S02_off_move'  
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_move'  
#rawname_ = 'S03_off_hold'  

rawname = rawname_ + '_resample_raw.fif'
#rawname = rawname_ + '_resample_afterICA_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    

In [ ]:
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
if 'projs' in raw.info:
    raw.info['projs'] = []

In [ ]:
import globvars as gv

In [ ]:
rectconv_fname_full = os.path.join(gv.data_dir, '{}_emg_rectconv.fif'.format(rawname_) )
rectconvraw  = mne.io.read_raw_fif(rectconv_fname_full)

In [ ]:
anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
anns = mne.read_annotations(anns_fn_full)

rectconvraw.set_annotations(anns)

In [ ]:
anns_MEGartif_fn = rawname_ + '_ann_MEGartif.txt'
anns_MEGartif_fn_full = os.path.join(data_dir, anns_MEGartif_fn)
anns_MEGartif = mne.read_annotations(anns_MEGartif_fn_full)

rectconvraw.annotations.append(anns_MEGartif.onset, anns_MEGartif.duration, anns_MEGartif.description)

In [ ]:
subj,medcond,task  = utils.getParamsFromRawname(rawname_)
sind_str = subj
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)

In [ ]:
maintremside = gen_subj_info[subj]['tremor_side']
print(maintremside)
#emg_ps = utils.getEMGperHand(rectconvraw)
emg_rot_ps = utils.getMainEMGcomp(rectconvraw)
emg = emg_rot_ps[maintremside]

megraw = raw.copy()
megraw.load_data()
megraw.pick_types(meg=True)

In [ ]:
task_start = utils.findTaskStart(rectconvraw.annotations)
task_offset = 5 # in sec
megraw.crop(tmin=0, tmax = task_start - task_offset)
emg.crop(tmin = 0, tmax = task_start - task_offset)

In [ ]:
task_start - task_offset

In [ ]:
from mne.decoding import CSP
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_predict

from mne import Epochs

In [ ]:
megraw.set_annotations(emg.annotations)

In [ ]:
#emg.set_annotations(megraw.annotations)

In [ ]:
print( megraw.info['bads'], emg.info['bads'] )
print(megraw.annotations, emg.annotations)

In [ ]:
%matplotlib qt
emg.plot(scalings={'eeg':'auto'});

In [ ]:
assert 0 == len(emg.times) - len(megraw.times)

In [ ]:
megraw.info['bads']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.decoding import CSP

In [ ]:
X = megraw.get_data().T

In [ ]:
X.shape

In [ ]:
mts_letter = gen_subj_info[subj]['tremor_side'][0].upper()


In [ ]:
sides_hand = [mts_letter]
int_types_basic = ['trem', 'notrem', 'hold', 'move']

In [ ]:
import utils_tSNE as utsne
sfreq = int(emg.info['sfreq'])

In [ ]:
ivalis = utils.ann2ivalDict(emg.annotations)
ivalis_tb, ivalis_tb_indarrays = utsne.getAnnBins(ivalis, emg.times, 
                                                  0, sfreq, 1, 
                                                  1, dataset_bounds=None)


In [ ]:
int_types_L = ['trem_L', 'notrem_L', 'hold_L', 'move_L', 'undef_L', 'holdtrem_L', 'movetrem_L']
int_types_R = ['trem_R', 'notrem_R', 'hold_R', 'move_R', 'undef_R', 'holdtrem_R', 'movetrem_R']
class_ids = {}
for ind, it in enumerate(int_types_L):
    class_ids[it] = ind+1
for ind, it in enumerate(int_types_R):
    class_ids[it] = -ind-1


In [ ]:
help(events_from_annotations)

In [ ]:
ivalis_tb_indarrays

In [ ]:
raw.annotations

In [ ]:
anns

In [ ]:
evt_dur_s = 1. # orig 0.25
epoch_dur_s = 2   # orig 1.5
decim = 1 # 8

In [ ]:
raw.load_data()

In [ ]:
evdict = {'notrem_{}'.format(mts_letter):1,'trem_{}'.format(mts_letter):2 }
events, event_id = events_from_annotations(rectconvraw, event_id=evdict, chunk_duration=evt_dur_s)

In [ ]:
megraw.load_data()

In [ ]:
# # Build epochs as sliding windows over the continuous raw file
# events = mne.make_fixed_length_events(megraw, id=1, duration=evt_dur_s)

In [ ]:
#events

In [ ]:
np.diff(events[:,0])

In [ ]:
meg_epochs = Epochs(megraw, events, tmin=0., tmax=epoch_dur_s, baseline=None,
                    detrend=0, decim=decim)

In [ ]:
meg_epochs

In [ ]:
meg_epochs.load_data()

In [ ]:
labels = meg_epochs.events[:, -1] - 1
len(labels)

In [ ]:
all_byhand= 0
if all_byhand:
    defclass = 0
    class_labels = np.repeat(defclass,len(X))
    assert defclass == 0

    old_ver = 0
    if old_ver:
        int_types = set()
        for itb in int_types_basic:
            for side in sides_hand:
                assert len(side) == 1
                int_types.update(['{}_{}'.format(itb,side)])
        #int_types = ['trem_L', 'notrem_L', 'hold_L', 'move_L']
        int_types = list(int_types)
        #print(int_types)

        classes = [k for k in ivalis_tb_indarrays.keys() if k in int_types]  #need to be ordered
        #classes

        for i,k in enumerate(classes):
            #print(i,k)
            for bininds in ivalis_tb_indarrays[k]:
                #print(i,len(bininds), bininds[0], bininds[-1])
                class_labels[ bininds ] = i + 1

    for itb in int_types_basic:
        for side in sides_hand:
            class_name = '{}_{}'.format(itb,side)
            if class_name not in ivalis_tb_indarrays:
                continue
            for bininds in ivalis_tb_indarrays[class_name]:
                #print(i,len(bininds), bininds[0], bininds[-1])
                class_labels[ bininds ] = class_ids[class_name]


    class_labels_good = class_labels

    rem_neut = 1
    if rem_neut:
        neq = class_labels_good != defclass
        inds = np.where( neq)[0]
        X_good = X[inds]
        class_labels_good = class_labels_good[inds]
    else:
        classes = ['neut'] + classes  # will fail if run more than once

    assert len(labels) == len(X_good)

    labels = class_labels_good

In [ ]:
help(ShuffleSplit)

In [ ]:
meg_epochs_train  = meg_epochs.copy()
meg_epochs_train.load_data()
meg_epochs_train.crop(tmin=0,tmax=epoch_dur_s/2)

In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = meg_epochs.get_data() #epochs.get_data()
epochs_data_train = meg_epochs_train.get_data() #epochs_train.get_data()

In [ ]:
#help(CSP)

In [ ]:
cv = ShuffleSplit(10, test_size=0.2, random_state=0)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])

In [ ]:
epochs_data.shape

In [ ]:
labels.shape

In [ ]:
# I need to generate fixed length events and then epochs, label them as tremor or not and then apply stuff

In [ ]:
scores = cross_val_score(clf, epochs_data_train, labels,cv=cv, n_jobs=10)

In [ ]:
#scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

In [ ]:
csp.plot_patterns(meg_epochs.info, ch_type='mag', size=1.5)

In [ ]:
csp.plot_patterns(meg_epochs.info, ch_type='grad', size=1.5)

# Over time

In [ ]:
sfreq = raw.info['sfreq']
w_length = int(sfreq * 0.5)   # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
    X_test = csp.transform(epochs_data_train[test_idx])

    # fit classifier
    lda.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    for n in w_start:
        X_test = csp.transform(epochs_data[test_idx][:, :, n:(n + w_length)])
        score_this_window.append(lda.score(X_test, y_test))
    scores_windows.append(score_this_window)

In [ ]:
# Plot scores over time
w_times = (w_start + w_length / 2.) / sfreq + meg_epochs.tmin

plt.figure()
plt.plot(w_times, np.mean(scores_windows, 0), label='Score')
plt.axvline(0, linestyle='--', color='k', label='Onset')
plt.axhline(0.5, linestyle='-', color='k', label='Chance')
plt.xlabel('time (s)')
plt.ylabel('classification accuracy')
plt.title('Classification score over time')
plt.legend(loc='lower right')
plt.show()